In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 74.0 MB/s eta 0:00:00


In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import rasterio
from sklearn.model_selection import train_test_split

# --- Config ---
IMAGE_DIR = "/content/drive/My Drive/HLS_Composite_Patches_2021"
LABEL_CSV = "/content/drive/My Drive/HLS_Composite_Patches_2021/HLS_Patch_Labels_2021.csv"
IMG_SIZE = (256, 256)
BATCH_SIZE = 32
EPOCHS = 30

# --- 1. Load label mapping ---
labels_df = pd.read_csv(LABEL_CSV)
labels_dict = dict(zip(labels_df["fileNamePrefix"], labels_df["label"]))

# --- 2. Collect image paths and match to labels ---
all_image_paths = [os.path.join(IMAGE_DIR, f) for f in os.listdir(IMAGE_DIR) if f.endswith('.tif')]

# Extract filename without extension to match CSV
filtered_paths = [
    p for p in all_image_paths
    if os.path.splitext(os.path.basename(p))[0] in labels_dict
]

print(f"Matched {len(filtered_paths)} images with labels.")

Matched 1650 images with labels.


In [4]:
# --- 3. Split train/test ---
train_paths, test_paths = train_test_split(filtered_paths, test_size=0.2, random_state=288)

# --- 4. Load function: RGB image + scalar label ---
def load_image_and_label(path):
    path_str = path.decode()
    with rasterio.open(path_str) as src:
        img = src.read(out_shape=(3, *IMG_SIZE))  # (3, H, W)
        img = np.transpose(img, (1, 2, 0))  # to (H, W, 3)
        img = np.nan_to_num(img.astype(np.float32))

    basename = os.path.splitext(os.path.basename(path_str))[0]
    label = np.float32(labels_dict[basename])

    return img, label

# --- 5. Create tf.data.Dataset ---
def create_dataset(paths, batch_size=BATCH_SIZE, shuffle=True):
    def _parse(path):
        image, label = tf.numpy_function(load_image_and_label, [path], [tf.float32, tf.float32])
        image.set_shape([*IMG_SIZE, 3])
        label.set_shape([])
        return image, label

    ds = tf.data.Dataset.from_tensor_slices(paths)
    ds = ds.map(_parse, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(100)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

In [5]:
import matplotlib.pyplot as plt

train_ds = create_dataset(train_paths)
test_ds = create_dataset(test_paths, shuffle=False)

print("✅ Dataset ready.")

✅ Dataset ready.


In [6]:
# Test image
with rasterio.open(filtered_paths[0]) as src:
    print("Dtype:", src.dtypes)
    raw = src.read(1)  # first band
    print("Raw min/max:", raw.min(), raw.max())

Dtype: ('float64', 'float64', 'float64')
Raw min/max: -0.0026500000000000004 0.4601


In [7]:
from tensorflow.keras import layers, models

def build_image_regression_model(input_shape=(256, 256, 3)):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)

    x = layers.GlobalAveragePooling2D()(x)

    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(1)(x)

    return models.Model(inputs, outputs)


In [8]:
model = build_image_regression_model()

callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint("best_model.keras", save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6
    )
]

# --- Compile ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, clipnorm=1.0),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [9]:
# UPDATED IMAGE REG BASE MODEL
history = model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS, callbacks=callbacks)

Epoch 1/30
42/42 ━━━━━━━━━━━━━━━━━━━━ 616s 8s/step - loss: 875.6575 - root_mean_squared_error: 29.5609 - val_loss: 272.7048 - val_root_mean_squared_error: 16.5138 - learning_rate: 0.0010
Epoch 2/30
42/42 ━━━━━━━━━━━━━━━━━━━━ 356s 8s/step - loss: 184.5692 - root_mean_squared_error: 13.5469 - val_loss: 123.7397 - val_root_mean_squared_error: 11.1238 - learning_rate: 0.0010
Epoch 3/30
42/42 ━━━━━━━━━━━━━━━━━━━━ 346s 8s/step - loss: 137.4456 - root_mean_squared_error: 11.7149 - val_loss: 124.8257 - val_root_mean_squared_error: 11.1725 - learning_rate: 0.0010
Epoch 4/30
42/42 ━━━━━━━━━━━━━━━━━━━━ 385s 8s/step - loss: 134.9027 - root_mean_squared_error: 11.6109 - val_loss: 118.7033 - val_root_mean_squared_error: 10.8951 - learning_rate: 0.0010
Epoch 5/30
42/42 ━━━━━━━━━━━━━━━━━━━━ 341s 8s/step - loss: 136.3167 - root_mean_squared_error: 11.6638 - val_loss: 117.9212 - val_root_mean_squared_error: 10.8592 - learning_rate: 0.0010
Epoch 6/30
42/42 ━━━━━━━━━━━━━━━━━━━━ 385s 8s/step - loss: 130.63

In [10]:
loss, rmse = model.evaluate(test_ds)
print(f"Test RMSE: {rmse:.4f}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 40s 3s/step - loss: 115.5377 - root_mean_squared_error: 10.7377
Test RMSE: 10.7428


In [6]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

def build_resnet50_regression_model(input_shape=(256, 256, 3)):
    base = ResNet50(
        input_shape=input_shape,
        weights='imagenet',
        include_top=False  # Remove ImageNet classifier head
    )

    x = layers.GlobalAveragePooling2D()(base.output)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    output = layers.Dense(1)(x)  # Single scalar output

    model = models.Model(inputs=base.input, outputs=output)
    return model

In [7]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("best_model.keras", save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6
    )
]

model2 = build_resnet50_regression_model()

# --- Compile ---
model2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4, clipnorm=1.0),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [8]:
history2 = model2.fit(train_ds, validation_data=test_ds, epochs=20, callbacks=callbacks)

Epoch 1/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 624s 7s/step - loss: 621.5468 - root_mean_squared_error: 24.7757 - val_loss: 698.0421 - val_root_mean_squared_error: 26.4205 - learning_rate: 1.0000e-04
Epoch 2/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 147s 3s/step - loss: 110.0967 - root_mean_squared_error: 10.4808 - val_loss: 195.7522 - val_root_mean_squared_error: 13.9911 - learning_rate: 1.0000e-04
Epoch 3/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 205s 3s/step - loss: 60.7753 - root_mean_squared_error: 7.7936 - val_loss: 330.6154 - val_root_mean_squared_error: 18.1828 - learning_rate: 1.0000e-04
Epoch 4/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 129s 3s/step - loss: 62.1384 - root_mean_squared_error: 7.8713 - val_loss: 224.5216 - val_root_mean_squared_error: 14.9840 - learning_rate: 1.0000e-04
Epoch 5/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 162s 3s/step - loss: 48.1748 - root_mean_squared_error: 6.9384 - val_loss: 417.6788 - val_root_mean_squared_error: 20.4372 - learning_rate: 1.0000e-04
Epoch 6/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 201s 3s/step 

In [9]:
loss, rmse = model2.evaluate(test_ds)
print(f"Test RMSE: {rmse:.4f}")

11/11 ━━━━━━━━━━━━━━━━━━━━ 34s 3s/step - loss: 135.1998 - root_mean_squared_error: 11.6215
Test RMSE: 11.5026


In [10]:
# NEW TEST DATASETS
# --- Config ---
LOWER_DIR = "/content/drive/My Drive/HLS_Composite_Patches_2021_CRElower"
LOWER_LABEL_CSV = "/content/drive/My Drive/HLS_Composite_Patches_2021_CRElower/HLS_Patch_Labels_2021_lower.csv"

UPPER_DIR = "/content/drive/My Drive/HLS_Composite_Patches_2021_CREupper"
UPPER_LABEL_CSV = "/content/drive/My Drive/HLS_Composite_Patches_2021_CREupper/HLS_Patch_Labels_2021_upper.csv"


def map_test_set(IMAGE_DIR, LABEL_CSV):

  labels_df = pd.read_csv(LABEL_CSV)
  labels_dict = dict(zip(labels_df["fileNamePrefix"], labels_df["label"]))

  all_image_paths = [os.path.join(IMAGE_DIR, f) for f in os.listdir(IMAGE_DIR) if f.endswith('.tif')]

  # Extract filename without extension to match CSV
  filtered_paths = [
      p for p in all_image_paths
      if os.path.splitext(os.path.basename(p))[0] in labels_dict
  ]
  print(f"Matched {len(filtered_paths)} images with labels.")
  return (filtered_paths, labels_dict)

In [11]:
test_cre_lower = map_test_set(LOWER_DIR, LOWER_LABEL_CSV)
lower_dict = test_cre_lower[1]
test_cre_upper = map_test_set(UPPER_DIR, UPPER_LABEL_CSV)
upper_dict = test_cre_upper[1]

Matched 50 images with labels.
Matched 50 images with labels.


In [15]:
def load_image_and_label(path):
    path_str = path.decode()
    with rasterio.open(path_str) as src:
        img = src.read(out_shape=(3, *IMG_SIZE))  # (3, H, W)
        img = np.transpose(img, (1, 2, 0))  # to (H, W, 3)
        img = np.nan_to_num(img.astype(np.float32))

    basename = os.path.splitext(os.path.basename(path_str))[0]
    label = np.float32(upper_dict[basename])

    return img, label

def create_dataset(paths, batch_size=BATCH_SIZE, shuffle=True):
    def _parse(path):
        image, label = tf.numpy_function(load_image_and_label, [path], [tf.float32, tf.float32])
        image.set_shape([*IMG_SIZE, 3])
        label.set_shape([])
        return image, label

    ds = tf.data.Dataset.from_tensor_slices(paths)
    ds = ds.map(_parse, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(100)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

In [13]:
test_lower = create_dataset(test_cre_lower[0], shuffle=False)
# loss, rmse = model.evaluate(test_lower)
# print(f"Test RMSE (base model, lower bound): {rmse:.4f}")

In [16]:
test_upper = create_dataset(test_cre_upper[0], shuffle=False)
# loss, rmse = model.evaluate(test_upper)
# print(f"Test RMSE (base model, upper bound): {rmse:.4f}")

In [ ]:
loss, rmse = model.evaluate(test_upper)
print(f"Test RMSE (base model, upper bound): {rmse:.4f}")

In [14]:
loss, rmse = model2.evaluate(test_lower)
print(f"Test RMSE (ResNet model, lower bound): {rmse:.4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 20s 9s/step - loss: 232.6067 - root_mean_squared_error: 15.2490
Test RMSE (ResNet model, lower bound): 15.0575


In [17]:
loss, rmse = model2.evaluate(test_upper)
print(f"Test RMSE (ResNet model, upper bound): {rmse:.4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 15s 6s/step - loss: 62.1921 - root_mean_squared_error: 7.8669
Test RMSE (ResNet model, upper bound): 8.2570
